## 설치

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 14.9 MB/s 
     |████████████████████████████████| 7.6 MB 69.2 MB/s 
     |████████████████████████████████| 163 kB 88.8 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 15.2 MB/s 
     |████████████████████████████████| 212 kB 89.8 MB/s 
     |████████████████████████████████| 115 kB 76.8 MB/s 
     |████████████████████████████████| 127 kB 84.7 MB/s 
     |████████████████████████████████| 115 kB 84.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 14.6 MB/s 
     |████████████████████████████████| 166 kB 77.7 MB/s 
     |████████████████████████████████| 182 kB 85.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 166 kB 86.7 MB/s 
     |████████████████████████████████| 162 kB 89.2 MB/s 
     |████████████████████████████████| 162 kB 87.2 MB/s 
     |████████████████████████████████| 158 kB 87.5 MB/s 
     |████████████████████████████████| 157 kB 92.1 MB/s 
     |████████████████████████████████| 157 kB 90.0 MB/s 
     |████████████████████████████████| 157 kB 89.0 MB/s 
     |████████████████████████████████| 157 kB 88.6 MB/s 
     |████████████████████████████████| 157 kB 89.5 MB/s 
     |████████████████████████████████| 157 kB 91.2 MB/s 
     |████████████████████████████████| 157 kB 85.9 MB/s 
     |████████████████████████████████| 156 kB 89.0 MB/s 


In [ ]:
import torch
import torch.optim as optim

import numpy as np
import tqdm as tqdm

from transformers import(
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_constant_schedule,
    AdamW
)

In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## ReadCsv

In [ ]:
import pandas as pd
data = pd.DataFrame()
for i in range(1,7):
    temp_data = pd.read_csv(f'https://raw.githubusercontent.com/cmj-dev/groomProject/main/data/gen_train_data_{i}.csv')
    data = pd.concat([data,temp_data])

In [ ]:
data

,masked,original
0,쉴드 아니라 국가가해준거야.,쉴드가 아니라 국가가 면제해준거야.
1,짐승 들어ㅋㅋ,짐승입장도 들어봐야지ㅋㅋ
2,깐라고기까지,깐부라고 부르기까지 하다니
3,어머니 간 해 주기 싫어서 도망쳤다고...?,어머니 간 이식 해 주기 싫어서 도망쳤다고...?
4,##한 기념 내가 사 줄게,고생한 기념으로 내가 저녁 사 줄게
...,...,...
15529,옷 사고 속옷도까 돈이 하나도 안 남네,옷도 사고 속옷도 사니까 돈이 하나도 안 남네
15530,인스타 그렇게 이쁘고 몸매애들이 넘치는데 밖에 나오면 그런애들 단 한명도 찾기 힘드냐,인스타에 그렇게 이쁘고 몸매 좋은애들이 넘치는데 밖에 나오면 그런애들 단 한명도 찾...
15531,그러게 장 다 담가먹는 줄 …. 가서 사기만 하면 되는 걸,그러게 본인은 장 다 담가먹는 줄…. 마트 가서 사기만 하면 되는 걸
15532,하면,오죽 하면 그러겠나


In [ ]:
data = data.sample(frac=1,random_state=43)

In [ ]:
spliter = int(len(data)*0.8)

In [ ]:
train_df = data.iloc[:spliter,:]
dev_df = data.iloc[spliter:,:]

In [ ]:
train_df

,masked,original
1459,방돼,방 값이 좀 부담돼
5356,누가? 어디 한번 해보시든가...,누가 무섭대? 어디 한번 해보시든가...
2034,[UNK]는재도 포함 전라도만 가네,[UNK] 피디는 만재도 포함 전라도만 가네
6355,너 정치 모르지 않아..? 누구 나오는지는 알고있냐? ㅋㅋ,너 정치 잘 모르지 않아..? 누구누구 나오는지는 알고있냐?ㅋㅋ
140,여사친이랑 연락하지 말 몰래 연락하는거야,여사친이랑 연락하지 말랬더니 몰래 연락하는거야
...,...,...
4343,너 음료 다거면 뭐하러 사먹냐,너 음료 다 남길거면 뭐하러 사먹냐
11387,무시한데?? 그래도 아빠 엄마가 애 생각하는건 팩트아니야??,누가 무시한데?? 그래도 아빠보단 엄마가 애 생각하는건 팩트아니야??
2575,##기엔 졸 어떡해요,그러기엔 너무 졸린데 어떡해요
6797,인종는 하지,그래도 인종비하는 하지 말자


In [ ]:
dev_df

,masked,original
7362,[UNK] 이 서해 도발을 했네,[UNK]이 또 서해에서 도발을 했네
7045,관리하기 부스럽고 힘들 것 같음,가정에서 관리하기 부잡스럽고 힘들 것 같음
2871,방치 140명이나 접종받았대. 어이없다.,방치된 백신으로 140명이나 접종받았대. 어이없다.
6410,NaN,숨 참으셈
2012,NaN,구글에서 찾아봐라
...,...,...
11928,노력이 없잖아,그래도 너무 노력이 없잖아.
4627,##으로 들어온 애,이번에 신입으로 들어온 애 말이야
3714,[UNK] 계란은 한 3990원 하네 개꿀.,[UNK] 계란은 한 판에 3990원 하네 개꿀.
7437,그런 너는석?,그런 너는 1등석 타 봤냐?


In [ ]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_df)
dev_data = Dataset.from_pandas(dev_df)

In [ ]:
train_data

Dataset({
    features: ['masked', 'original', '__index_level_0__'],
    num_rows: 50620
})

## Tokenize

In [ ]:
model_name = 'skt/kogpt2-base-v2'
max_length = 256

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                          pad_token='<pad>', mask_token='<mask>')

In [ ]:
def tokenizeWithLabel(data):
    tokenized_datas = tokenizer(
        f"<unused0> <unused1> {data['masked']} <unused2> {data['original']} <unused3>",
        max_length=max_length,
        padding="max_length"
    )
    tokenized_datas['labels']=tokenized_datas["input_ids"]
    return tokenized_datas

In [ ]:
train_tokenized_datasets = train_data.map(tokenizeWithLabel, remove_columns=train_data.column_names)
dev_tokenized_datasets = dev_data.map(tokenizeWithLabel, remove_columns=dev_data.column_names)

  0%|          | 0/50620 [00:00<?, ?ex/s]

  0%|          | 0/12655 [00:00<?, ?ex/s]

In [ ]:
train_tokenized_datasets[0]

## Train

In [ ]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'eval/loss'},
    'parameters': 
    {
        'batch_size': {'values': [256]},
        'epochs': {'values': [5]},
        'lr': {'values': [1e-5]},
        'scheduler': {'values': ['linear', 'cosine', 'constant']}
     }
}

In [ ]:
max_batch_size = 32
def train():
    torch.cuda.empty_cache()
    model = AutoModelForCausalLM.from_pretrained(model_name)
    grouped_params = model.parameters()
    run = wandb.init(config=sweep_configuration, entity="groom2team")
    batch_size = wandb.config.batch_size if wandb.config.batch_size < max_batch_size else max_batch_size
    gradient_accumulation_steps= wandb.config.batch_size // max_batch_size
    epochs = wandb.config.epochs
    total_steps = int(len(train_tokenized_datasets)/wandb.config.batch_size*epochs)
    learning_rate = wandb.config.lr
    data_collator = default_data_collator
    grouped_params = model.parameters()
    optimizer=AdamW(grouped_params, lr=learning_rate)
    scheduler_type = wandb.config.scheduler
    if scheduler_type == 'linear':
        scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                                    num_warmup_steps=0,
                                                    num_training_steps=total_steps)
    elif scheduler_type == 'cosine':
        scheduler=get_cosine_schedule_with_warmup(optimizer=optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)
    elif scheduler_type == 'constant':
        scheduler=get_constant_schedule(optimizer=optimizer)
    optimizers = optimizer, scheduler
    args = TrainingArguments(
        f"{model_name}-finetuned",
        evaluation_strategy = "steps",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        report_to="wandb",
        run_name="utopia",
        logging_steps = total_steps//200,
        eval_steps = total_steps//100,
        save_steps = total_steps//100,
        weight_decay=0.0,
        save_total_limit = 2,
        load_best_model_at_end=True
    )
    trainer = Trainer(
        model,
        args,
        train_dataset=train_tokenized_datasets,
        eval_dataset=dev_tokenized_datasets,
        data_collator=data_collator,
        tokenizer=tokenizer,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
        optimizers=optimizers
    )
    trainer.train()# train 하고
    trainer.save_model(output_dir= 'pytorch_finetuned') # trainer에서 실행된 model save
    artifact = wandb.Artifact(name='pytorch_finetuned', type='model') # wandb에 해당 모델 version 관리.
    artifact.add_dir('pytorch_finetuned', name='best_model_at_end')
    run.log_artifact(artifact)

In [ ]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project='pj3_gen_gpt2', entity='groom2team')
count = 3

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 4wgtghgb
Sweep URL: https://wandb.ai/groom2team/pj3_gen_gpt2/sweeps/4wgtghgb


In [ ]:
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: x1dzbcb7 with config:
wandb: 	batch_size: 256
wandb: 	epochs: 5
wandb: 	lr: 1e-05
wandb: 	scheduler: linear
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 50620
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 985
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Run x1dzbcb7 errored: ValueError('expected sequence of length 14 at dim 1 (got 25)')
wandb: ERROR Run x1dzbcb7 errored: ValueError('expected sequence of length 14 at dim 1 (got 25)')
wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
wandb.finish() # wandb 종료